<a href="https://colab.research.google.com/github/rnop/WSB-Top-Tickers-and-Sentiment-Analysis-Project/blob/main/scrape_comments_tickers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
if os.path.isfile("/content/scraper_utils.py"):
  print("'nasdaq_tickers.csv' is already downloaded.") 
else:
  ! wget https://raw.githubusercontent.com/rnop/WSB-Top-Tickers-and-Sentiment-Analysis-Project/main/scraper_utils.py # download CSV

--2021-12-10 16:18:49--  https://raw.githubusercontent.com/rnop/WSB-Top-Tickers-and-Sentiment-Analysis-Project/main/scraper_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3342 (3.3K) [text/plain]
Saving to: ‘scraper_utils.py’

scraper_utils.py    100%[===================>]   3.26K  --.-KB/s    in 0s      

2021-12-10 16:18:49 (40.2 MB/s) - ‘scraper_utils.py’ saved [3342/3342]



In [ ]:
! pip install praw

     |████████████████████████████████| 176 kB 5.1 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 


In [ ]:
from scraper_utils import * 
import pandas as pd
import praw 
import re 
import requests
import json
import csv
import time
import datetime

### Scrape Reddit Comments

In [ ]:
# Connect to Reddit API via praw
reddit_creds = praw.Reddit(client_id='***', client_secret='***', user_agent='***', check_for_async=False)
# Obtain URLs of Daily Submission Threads within time period
urls = get_daily_discussion_urls(subreddit='wallstreetbets', before_date= str(datetime.datetime.now())[:10], after_date='2021-01-01')
# Scrape Comments Using Each URL
comments = get_comments(urls, reddit_creds)
# Read Comments into DataFrame
comments_df = comments_to_df(comments, urls)

In [ ]:
# Clean comments so it is easier to extract tickers
comments_df['comment'] = comments_df.apply(lambda x: x['comment'].replace('.', ' '), axis=1)
comments_df['comment'] = comments_df.apply(lambda x: x['comment'].replace(',', ' '), axis=1)
comments_df['comment'] = comments_df.apply(lambda x: x['comment'].replace("'", ' '), axis=1)

# Some punctuations are actually useful for the sentiment model
comments_df['comment'] = comments_df.apply(lambda x: x['comment'].replace('$', ' $ '), axis=1)
comments_df['comment'] = comments_df.apply(lambda x: x['comment'].replace('#', ' # '), axis=1) 
comments_df['comment'] = comments_df.apply(lambda x: x['comment'].replace("?", ' ? '), axis=1)
comments_df['comment'] = comments_df.apply(lambda x: x['comment'].replace("!", ' ! '), axis=1)

In [ ]:
print(comments_df.info())
comments_df.sample(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 224241 entries, 0 to 982
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   comment  224241 non-null  object        
 1   date     224241 non-null  datetime64[ns]
 2   title    224241 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 6.8+ MB
None


,comment,date,title
323,APHA is merging with TLRY in Q2 2021 For ever...,2021-02-09,"Daily Discussion Thread for February 09, 2021"
316,what happened with everyone being bulls every...,2021-09-14,"Daily Discussion Thread for September 14, 2021"
853,Holy shit i bought SPY at 442 80 this morning ...,2021-08-12,"Daily Discussion Thread for August 12, 2021"
295,[removed],2021-09-02,"Daily Discussion Thread for September 02, 2021"
417,Only lost 3k this week 😎,2021-11-12,"Daily Discussion Thread for November 12, 2021"
55,[deleted],2021-09-02,"Daily Discussion Thread for September 02, 2021"
388,Don’t be scared DIS the green isn’t gonna hurt...,2021-11-29,"Daily Discussion Thread for November 29, 2021"
305,Y’all really bought TSLA calls,2021-07-26,"Daily Discussion Thread for July 26, 2021"
206,Honestly I don’t understand why I buy anythin...,2021-09-23,"Daily Discussion Thread for September 23, 2021"
122,See that big red SPY dildo ? That’s exactly w...,2021-10-27,"Daily Discussion Thread for October 27, 2021"


In [ ]:
comments_df.head()

,comment,date,title
0,What am I even supposed to do with myself for ...,2021-01-01,"Daily Discussion Thread for January 01, 2021"
1,Goddamn if there’s one sub I can go on to find...,2021-01-01,"Daily Discussion Thread for January 01, 2021"
2,I usually ignore most of the shit people say o...,2021-01-01,"Daily Discussion Thread for January 01, 2021"
3,“Sorry honey can’t stay up for midnight marke...,2021-01-01,"Daily Discussion Thread for January 01, 2021"
4,What if 2020 is so inescapable that today is a...,2021-01-01,"Daily Discussion Thread for January 01, 2021"


### Get Stock Tickers

In [ ]:
import os
if os.path.isfile("/content/nasdaq_tickers.csv"):
  print("'nasdaq_tickers.csv' is already downloaded.") 
else:
  ! wget https://raw.githubusercontent.com/rnop/WSB-Top-Tickers-and-Sentiment-Analysis-Project/main/nasdaq_tickers.csv # download CSV

--2021-12-10 17:27:37--  https://raw.githubusercontent.com/rnop/WSB-Top-Tickers-and-Sentiment-Analysis-Project/main/nasdaq_tickers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 618348 (604K) [text/plain]
Saving to: ‘nasdaq_tickers.csv’

nasdaq_tickers.csv  100%[===================>] 603.86K  --.-KB/s    in 0.04s   

2021-12-10 17:27:38 (14.1 MB/s) - ‘nasdaq_tickers.csv’ saved [618348/618348]



In [ ]:
tickers = pd.read_csv('nasdaq_tickers.csv')

# Clean Last Sale column for filtering
tickers['Last Sale'] = pd.to_numeric(tickers['Last Sale'].str.replace('$', ''))

# Filter tickers based on price of shares, market cap, and country
tickers = tickers[(tickers['Last Sale'] > 5.00) & 
                  (tickers['Market Cap'] > 100000000) &
                  (tickers['Country'].isin(['United States', 'Canada']))]

tickers = tickers[['Symbol', 'Name', 'Sector']]
tickers = tickers.append({'Symbol': 'SPY', 'Name':'S&P500', 'Sector':'ETF'}, ignore_index=True)

# Add tickers, company name, and sector
tickers = tickers.append({'Symbol': 'PLTR', 'Name':'Palantir', 'Sector':'Technology'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'GME', 'Name':'Gamestop', 'Sector':'Meme'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'AMC', 'Name':'AMC', 'Sector':'Meme'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'BB', 'Name':'BlackBerry', 'Sector':'Meme'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'CLOV', 'Name':'Clover Health', 'Sector':'Meme'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'CHWY', 'Name':'Chewy', 'Sector':'Meme'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'SPCE', 'Name':'Virgin Galactic', 'Sector':'Meme'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'NOK', 'Name':'Nokia', 'Sector':'Meme'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'NAKD', 'Name':'Naked Brands', 'Sector':'Meme'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'CCL', 'Name':'Carnival Cruise Line', 'Sector':'Meme'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'NCLH', 'Name':'Norwegian Cruise Line', 'Sector':'Meme'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'RKT', 'Name':'Rocket Insurance', 'Sector':'Meme'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'X', 'Name':'US Steel Corporation', 'Sector':'Energy'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'WISH', 'Name':'WISH', 'Sector':'Meme'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'JPM', 'Name':'JP Morgan', 'Sector':'Finance'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'BAC', 'Name':'Bank of America', 'Sector':'Finance'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'V', 'Name':'Visa', 'Sector':'Finance'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'PFE', 'Name':'Pfizer', 'Sector':'Health Care'}, ignore_index=True)


tickers = tickers.append({'Symbol': 'DIS', 'Name':'Disney', 'Sector':'Consumer Services'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'BABA', 'Name':'Alibaba', 'Sector':'Consumer Durables'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'JD', 'Name':'JD', 'Sector':'Consumer Durables'}, ignore_index=True)
tickers = tickers.append({'Symbol': 'NIO', 'Name':'NIO', 'Sector':'Technology'}, ignore_index=True)

# Remove commonly mistakened tickers made by users
mistakened_tickers = ['ON', 'ME', 'GO','HAS','GOOD','OPEN','LOVE','LMAO','EVER','PLAY','REAL','LIFE','RUN','VERY',
                      'CASH','HOPE','CAR','FREE','GAIN','GLAD','COOL','KIDS','CARE','HEAR', 'APP', 'STEP', 'TECH',
                      'PLUS','FUND','FAT','RENT', 'POW', 'TIL','JACK','MON','ROLL']

tickers = tickers[~tickers['Symbol'].isin(mistakened_tickers)]

tickers.sample(5)

,Symbol,Name,Sector
361,CCD,Calamos Dynamic Convertible & Income Fund Comm...,Finance
1732,ROAD,Construction Partners Inc. Class A Common Stock,Capital Goods
1933,STRO,Sutro Biopharma Inc. Common Stock,Health Care
1750,RRR,Red Rock Resorts Inc. Class A Common Stock,Consumer Services
1269,MCMJ,Merida Merger Corp. I Common Stock,NaN


## Search for Tickers in Comments

In [ ]:
ticker_in_comment = []
for comment in comments_df['comment']:
  all_tickers_in_comment = []
  for word in comment.split():
    if word.upper() in tickers['Symbol'].values:
      all_tickers_in_comment.append(word.upper())
  # ticker_in_comment.append(all_tickers_in_comment)
  ticker_in_comment.append(list(set(all_tickers_in_comment))) # some users write the ticker multiple times in their post

comments_df['ticker_in_comment'] = ticker_in_comment

In [ ]:
comments_df.to_csv('submission_comments_tickers.csv', index=False)